In [9]:
import pandas as pd
from pathlib import Path
from concurrent.futures import ThreadPoolExecutor, as_completed
from aspiratio.utils.report_downloader import find_annual_reports

repo = Path('..').resolve()
coverage_path = repo / 'coverage_table_updated.csv'
validation_path = repo / 'validation_results.csv'

coverage = pd.read_csv(coverage_path, sep='\t')
years = list(range(2019, 2025))

# Load validation results to identify already-validated reports
validated_df = pd.read_csv(validation_path)
validated_set = set(
    (row['CID'], row['Year']) 
    for _, row in validated_df[validated_df['Valid'] == True].iterrows()
)
print(f"✓ Already validated: {len(validated_set)} company-year combinations")

# Filter to incomplete entries (not Complete ✓)
incomplete = coverage[coverage['Priority'] != 'Complete ✓']

# Further filter out any company-year combos that are already validated
def needs_search(row):
    return (row['Company_Identifier'], row['FiscalYear']) not in validated_set

incomplete_needing_search = incomplete[incomplete.apply(needs_search, axis=1)]
print(f"→ Incomplete in coverage: {len(incomplete)}")
print(f"→ Actually need to search: {len(incomplete_needing_search)}")

incomplete_needing_search[['CompanyName', 'Company_Identifier', 'FiscalYear', 'IR_URL']].head(10)

✓ Already validated: 18 company-year combinations
→ Incomplete in coverage: 195
→ Actually need to search: 159


,CompanyName,Company_Identifier,FiscalYear,IR_URL
6,Atlas Copco A,S6,2019,https://www.atlascopcogroup.com/en/investors
8,Atlas Copco A,S6,2021,https://www.atlascopcogroup.com/en/investors
9,Atlas Copco A,S6,2022,https://www.atlascopcogroup.com/en/investors
10,Atlas Copco A,S6,2023,https://www.atlascopcogroup.com/en/investors
12,Boliden,S7,2019,https://investors.boliden.com/
13,Boliden,S7,2020,https://investors.boliden.com/
14,Boliden,S7,2021,https://investors.boliden.com/
15,Boliden,S7,2022,https://investors.boliden.com/
16,Boliden,S7,2023,https://investors.boliden.com/
17,Boliden,S7,2024,https://investors.boliden.com/


In [10]:
# Build search tasks: group by company and collect missing years
search_tasks = {}
for company, group in incomplete_needing_search.groupby(['CompanyName', 'Company_Identifier', 'IR_URL']):
    name, cid, ir_url = company
    target_years = sorted(group['FiscalYear'].unique().tolist())
    search_tasks[(name, cid, ir_url)] = target_years

print(f"🔍 Search tasks: {len(search_tasks)} companies")
for (name, cid, _), yrs in list(search_tasks.items())[:5]:
    print(f"   {name} ({cid}): years {yrs}")

🔍 Search tasks: 25 companies
   Alfa Laval (S3): years [2019, 2020, 2021, 2022, 2023, 2024]
   Atlas Copco A (S6): years [2019, 2021, 2022, 2023]
   Boliden (S7): years [2019, 2020, 2021, 2022, 2023, 2024]
   EQT (S9): years [2019, 2020, 2021, 2022, 2023, 2024]
   Epiroc A (S8): years [2019, 2020, 2021, 2022, 2023, 2024]


In [11]:
# Search for annual reports in parallel with MFN fallback
import importlib
import aspiratio.utils.report_downloader as rd_module
import aspiratio.tier1.mfn_search as mfn_module

# Reload modules to pick up any changes
importlib.reload(mfn_module)
importlib.reload(rd_module)

from aspiratio.utils.report_downloader import find_annual_reports
from aspiratio.tier1.mfn_search import find_reports_via_mfn

def search_company(args):
    """Worker function to search one company with MFN fallback."""
    name, cid, ir_url, target_years = args
    results = []
    
    # Try standard IR search first
    try:
        candidates = find_annual_reports(ir_url, years=target_years, company_name=name)
        for c in candidates:
            results.append({
                'Company_Identifier': cid,
                'CompanyName': name,
                'Year': c.get('year'),
                'URL': c.get('url'),
                'Source_Page': c.get('source_page', ir_url),
                'Title': c.get('title', ''),
                'Source': c.get('source', 'ir')
            })
    except Exception as e:
        print(f"    ⚠ IR search error for {name}: {e}")
    
    # Check which years are still missing
    found_years = {r['Year'] for r in results}
    missing_years = [y for y in target_years if y not in found_years]
    
    # If any years missing, try MFN fallback explicitly
    if missing_years:
        print(f"    🔄 {name}: Missing years {missing_years}, trying MFN fallback...")
        try:
            mfn_candidates = find_reports_via_mfn(name, missing_years)
            for c in mfn_candidates:
                results.append({
                    'Company_Identifier': cid,
                    'CompanyName': name,
                    'Year': c.get('year'),
                    'URL': c.get('url'),
                    'Source_Page': c.get('source_page', ''),
                    'Title': c.get('title', ''),
                    'Source': 'mfn'
                })
        except Exception as e:
            print(f"    ⚠ MFN fallback error for {name}: {e}")
    
    status = 'success' if results else 'no_results'
    return name, cid, status, results

# Prepare tasks
task_list = [
    (name, cid, ir_url, yrs) 
    for (name, cid, ir_url), yrs in search_tasks.items()
]

# Run searches in parallel (max 5 concurrent to avoid overwhelming servers)
report_candidates = []
max_workers = min(5, len(task_list)) if task_list else 1

print(f"🚀 Starting parallel search with {max_workers} workers...")
print(f"   Companies to search: {len(task_list)}")

with ThreadPoolExecutor(max_workers=max_workers) as executor:
    futures = {executor.submit(search_company, task): task for task in task_list}
    
    for future in as_completed(futures):
        name, cid, status, results = future.result()
        if results:
            print(f"  ✓ {name} ({cid}): found {len(results)} candidates")
        else:
            print(f"  ✗ {name} ({cid}): no candidates found")
        report_candidates.extend(results)

print(f"\n📊 Total candidates found: {len(report_candidates)}")
candidates_df = pd.DataFrame(report_candidates)
candidates_df.head(10)

🚀 Starting parallel search with 5 workers...
   Companies to search: 25
Searching: https://www.alfalaval.com/investors/
Searching: https://www.atlascopcogroup.com/en/investors
Searching: https://investors.boliden.com/
Searching: https://eqtgroup.com/shareholders/reports-and-presentations
Searching: https://www.epirocgroup.com/en/investors/investor-contacts
  → Found 1983 links to analyze (from HTML, JSON, nav, footer)
  Searching: https://www.epirocgroup.com/en/system/in-your-country
  → Found 2637 links to analyze (from HTML, JSON, nav, footer)
    ✗ Excluded: download quarterly report (Quarterly report)
    ✗ Excluded: download quarterly report (Quarterly report)
  → Analyzed 6 PDF links, excluded 2 quarterly/SEC filings
  Searching: https://www.atlascopcogroup.com/en/investors/reports-and-presentations
  → Found 608 links to analyze (from HTML, JSON, nav, footer)
  → Analyzed 330 PDF links, excluded 0 quarterly/SEC filings
  Searching: https://lp.eqtgroup.com/login
  → Found 2098 li

,Company_Identifier,CompanyName,Year,URL,Source_Page,Title,Source
0,S3,Alfa Laval,2024,https://mb.cision.com/Main/905/4128683/3359424...,https://mfn.se/cis/a/alfa-laval/alfa-laval-pub...,Annual Report 2024 (via MFN),mfn
1,S3,Alfa Laval,2024,https://mb.cision.com/Public/905/4128683/8e75a...,https://mfn.se/cis/a/alfa-laval/alfa-laval-pub...,Annual Report 2024 (via MFN),mfn
2,S3,Alfa Laval,2024,https://mb.cision.com/Main/905/4128721/3359435...,https://mfn.se/cis/a/alfa-laval/alfa-lavals-an...,Annual Report 2024 (via MFN),mfn
3,S3,Alfa Laval,2024,https://mb.cision.com/Public/905/4128721/b3609...,https://mfn.se/cis/a/alfa-laval/alfa-lavals-an...,Annual Report 2024 (via MFN),mfn
4,S3,Alfa Laval,2019,https://mb.cision.com/Main/905/3069338/1218261...,https://mfn.se/cis/a/alfa-laval/alfa-lavals-an...,Annual Report 2019 (MFN/Cision),mfn
5,S3,Alfa Laval,2020,https://mb.cision.com/Main/905/3318000/1395725...,https://mfn.se/cis/a/alfa-laval/alfa-laval-alf...,Annual Report 2020 (MFN/Cision),mfn
6,S3,Alfa Laval,2020,https://mb.cision.com/Main/905/3227250/1327349...,https://mfn.se/cis/a/alfa-laval/alfa-laval-sup...,Annual Report 2020 (MFN/Cision),mfn
7,S3,Alfa Laval,2020,https://mb.cision.com/Main/905/3220154/1322018...,https://mfn.se/cis/a/alfa-laval/alfa-laval-sup...,Annual Report 2020 (MFN/Cision),mfn
8,S3,Alfa Laval,2021,https://mb.cision.com/Main/905/3536813/1557177...,https://mfn.se/cis/a/alfa-laval/alfa-lavals-an...,Annual Report 2021 (MFN/Cision),mfn
9,S3,Alfa Laval,2021,https://mb.cision.com/Public/905/3536813/b3ca6...,https://mfn.se/cis/a/alfa-laval/alfa-lavals-an...,Annual Report 2021 (MFN/Cision),mfn


In [12]:
# Persist candidates for review (only if we found any)
if not candidates_df.empty:
    output = repo / 'report_candidates.csv'
    candidates_df.to_csv(output, index=False)
    print(f"✓ Saved {len(candidates_df)} candidates to {output}")
else:
    print("⚠ No candidates found - nothing to save")

✓ Saved 288 candidates to /Users/jakobjohannesson/Documents/github_repos/aspiratio/report_candidates.csv


In [8]:
# Test MFN fallback search for companies that failed regular search
# This uses mfn.se which aggregates Swedish company press releases with Cision PDF links

from aspiratio.tier1.mfn_search import find_reports_via_mfn
import importlib
import aspiratio.tier1.mfn_search as mfn_module

# Reload to pick up changes
importlib.reload(mfn_module)
from aspiratio.tier1.mfn_search import find_reports_via_mfn

# Test with Alfa Laval
test_company = "Alfa Laval"
test_years = [2023, 2024]

print(f"Testing MFN search for {test_company}, years {test_years}")
mfn_results = find_reports_via_mfn(test_company, test_years)

if mfn_results:
    print(f"\n✅ Found {len(mfn_results)} reports via MFN:")
    for r in mfn_results:
        print(f"  Year {r['year']}: {r['url'][:80]}...")
else:
    print("\n❌ No reports found via MFN")

Testing MFN search for Alfa Laval, years [2023, 2024]
  🌐 Trying MFN search for Alfa Laval...
  🔍 Trying MFN URL: https://mfn.se/all/a/alfa-laval
    ✓ Found MFN company page

  📡 Strategy 1: Scanning news feed...
  📡 Scanning MFN feed: https://mfn.se/all/a/alfa-laval
  📄 Extracting attachments from: https://mfn.se/cis/a/alfa-laval/alfa-laval-publicerar-ars-och-hallbarhetsredovisning-for-2024-f6a65811
    ✓ Found Cision PDF: https://mb.cision.com/Main/905/4128683/3359424.pdf
    ✓ Found Cision PDF: https://mb.cision.com/Public/905/4128683/8e75a2deaf874680.pdf
  📄 Extracting attachments from: https://mfn.se/cis/a/alfa-laval/alfa-lavals-annual-and-sustainability-report-for-2024-published-83349e40
    ✓ Found Cision PDF: https://mb.cision.com/Main/905/4128721/3359435.pdf
    ✓ Found Cision PDF: https://mb.cision.com/Public/905/4128721/b3609387bc96eb5f.pdf
    ✓ Found 4 annual reports from feed scan
    ✓ Found reports for years: [2024]

  🔍 Strategy 2: Searching for remaining years [2023]